```
[Title] Multivariate Multi Labels LSTM
[Author] Jason Brownlee
[Etidor] Yibeck.Lee(yibeck.lee@gmail.com)
[reference]
 - https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
```

In [11]:
# multivariate output stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [12]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)



In [13]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)

# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[[10 15 25]
 [20 25 45]
 [30 35 65]] [40 45 85]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [ 50  55 105]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [ 60  65 125]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [ 70  75 145]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [ 80  85 165]
[[ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]] [ 90  95 185]


In [14]:
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')


In [15]:
# fit model
model.fit(X, y, epochs=400, verbose=1)


Epoch 1/400
6/6 [==============================] - 4s 690ms/step - loss: 9327.6846
Epoch 2/400
6/6 [==============================] - 0s 2ms/step - loss: 9078.4648
Epoch 3/400
6/6 [==============================] - 0s 3ms/step - loss: 8836.1572
Epoch 4/400
6/6 [==============================] - 0s 5ms/step - loss: 8558.3711
Epoch 5/400
6/6 [==============================] - 0s 3ms/step - loss: 8296.7031
Epoch 6/400
6/6 [==============================] - 0s 4ms/step - loss: 8033.1743
Epoch 7/400
6/6 [==============================] - 0s 5ms/step - loss: 7739.2891
Epoch 8/400
6/6 [==============================] - 0s 4ms/step - loss: 7418.2446
Epoch 9/400
6/6 [==============================] - 0s 3ms/step - loss: 7056.1875
Epoch 10/400
6/6 [==============================] - 0s 3ms/step - loss: 6651.9917
Epoch 11/400
6/6 [==============================] - 0s 3ms/step - loss: 6236.7173
Epoch 12/400
6/6 [==============================] - 0s 4ms/step - loss: 5802.5093
Epoch 13/400
6/6 [=====

In [16]:
# demonstrate prediction
x_input = array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=1)


1/1 [==============================] - 1s 1s/step


In [17]:
yhat

array([[ 99.60699 , 105.548294, 204.85071 ]], dtype=float32)